In [4]:
import os
import gc
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
#from kaggle_datasets import KaggleDatasets
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
import cloudpickle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        #return_attention_masks=False, 
        return_token_type_ids=False,
        padding='max_length',
        pad_to_max_length=True,
        max_length=maxlen
    )
    res = [x[:maxlen] for x in enc_di['input_ids']]
    return np.asarray(res)

def build_model(transformer, max_len=512):
    """
    https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
    """
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='linear')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=9e-6), loss='mean_squared_error', metrics=['mean_squared_error'])
    
    return model

def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Error [MSE]')
    plt.legend()
    plt.grid(True)

In [7]:
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [8]:
AUTO = tf.data.experimental.AUTOTUNE
EPOCHS = 10
BATCH_SIZE = 4 * strategy.num_replicas_in_sync
MAX_LEN = 194
MODEL = '../input/jplu-tf-xlm-roberta-large'
downsample1 = None # for external pseudo-labeled data
downsample2 = 200 # for external pseudo-labeled data

In [9]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

400 Client Error: Bad Request for url: https://huggingface.co/../input/jplu-tf-xlm-roberta-large/resolve/main/config.json


OSError: Can't load config for '../input/jplu-tf-xlm-roberta-large'. Make sure that:

- '../input/jplu-tf-xlm-roberta-large' is a correct model identifier listed on 'https://huggingface.co/models'

- or '../input/jplu-tf-xlm-roberta-large' is the correct path to a directory containing a config.json file

